In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

In [ ]:
from qat.core.console import display
from qat.lang.AQASM import Program

## 1. Cargamos un qbit

Siguiendo el algoritmo de carga empezamos con un estado $|0\rangle$ sobre el que queremos carga la función de probabilidad $p(x)$. 

Para ello en el primer qbit lo que hacemos es:

$$|\Psi_1\rangle = \sqrt{p_{0}^{1}} |0\rangle + \sqrt{p_{1}^{1}} |1\rangle$$

Como solo tenemos un q-bit dividimos todo el dominio x en 2 trozos ($2^{m}, m=1$):

$$Dom(x) = I_0 \cup I_1$$

y por definición:

$$p_{0}^{1} = \int_{I_0}{p(x)dx}$$
$$p_{1}^{1} = \int_{I_1}{p(x)dx}$$

Esto se puede conseguir mediante una rotación alrededor del eje y de:

$$\theta_{0} = \arccos{\sqrt{p_{0}^{1}}}$$

$$|\Psi_1\rangle = \cos{\theta_{0}} |0\rangle + \sin{\theta_{0}} |1\rangle$$

Esto se puede conseguir particularizando a i=0 la siguiente función general:

$$f(i) = \frac{\int_{x_{L}^{i}}^{\frac{x_{R}^{i}-x_{L}^{i}}{2}}{p(x)dx}}{\int_{x_{L}^{i}}^{x_{R}^{i}}{p(x)dx}}$$

$$\theta_{i} = \arccos{\sqrt{f(i)}}$$

En el caso de un solo qbit:

$$f(0) = \frac{\int_{x_{L}^{0}}^{\frac{x_{R}^{0}-x_{L}^{0}}{2}}{p(x)dx}}{\int_{x_{L}^{0}}^{x_{R}^{0}}{p(x)dx}}=\frac{\int_{I_0}{p(x)dx}}{\int_{I_0 \cup I_1}{p(x)dx}} = \int_{I_0}{p(x)dx} = p_{0}^{1}$$

Donde hemos utilizado el hecho de que:

$$\int_{I_0 \cup I_1}{p(x)dx}=\int_{I_0}{p(x)dx}+\int_{I_1}{p(x)dx} = 1$$

La última igualdad solo es cierta si $I_0 \cup I_1$ es igual a todo el dominio de X

$$\theta_{0} = \arccos{\sqrt{f(0)}}$$
$$|\Psi_1\rangle = \cos{\theta_{0}} |0\rangle + \sin{\theta_{0}} |1\rangle$$

In [ ]:
def get_histogram(p, a, b, nbin):
	"""
	Given a function p, convert it into a histogram. The function must be positive, the normalization is automatic.
	Note that instead of having an analytical expression, p could just create an arbitrary vector of the right dimensions and positive amplitudes
	so that this procedure could be used to initialize any quantum state with real amplitudes
	
	a    (float)    = lower limit of the interval
	b    (float)    = upper limit of the interval
	p    (function) = function that we want to convert to a probability mass function. It does not have to be normalized but must be positive in the interval
	nbin (int)      = number of bins in the interval
	"""
	step = (b-a)/nbin
	centers = np.array([a+step*(i+1/2) for i in range(nbin)]) #Calcula directamente los centros de los bines

	prob_n = p(centers)
	assert np.all(prob_n>=0.), 'Probabilities must be positive, so p must be a positive function'
	probs = prob_n/np.sum(prob_n)
	assert np.isclose(np.sum(probs), 1.), 'Probability is not getting normalized properly'
	return centers, probs


In [ ]:
def p(x):
    return x*x

In [ ]:
nqbits = 1
centers, probs = get_histogram(p, 0, 1, 2**nqbits)

In [ ]:
centers

In [ ]:
probs

In [ ]:
plt.plot(np.linspace(0,1), p(np.linspace(0,1)), 'o')
plt.plot(centers, probs, 'o')

In [ ]:
qprog = Program()
qbits = qprog.qalloc(1)
f_0 = probs[0]/sum(probs)
print('f_0: {}'.format(f_0))
theta_0 = np.arccos(np.sqrt(f_0))
print('theta_0: {}'.format(theta_0))
from qat.lang.AQASM import RY
qprog.apply(RY(2*theta_0),qbits)


In [ ]:
#Create the circuit from the program
circuit = qprog.to_circ()

#Display the circuit
%qatdisplay circuit

In [ ]:
#Create a Job from the circuit
job = circuit.to_job()

#Import and create the linear algebra simulator
from qat.qpus import LinAlg
linalgqpu = LinAlg()

#Submit the job to the simulator LinAlg and get the results
result = linalgqpu.submit(job)

#Print the results
for sample in result:
    print("State %s probability %s" % (sample.state, sample.probability))

Ahora añadimos un qbit sobre el que realizaremos una operación contralada en función de si en el  qbit 0 tenemos un estado $|0\rangle$ ó un estado $|1\rangle$

$$U_f |\Psi_1\rangle \otimes  |0\rangle= \cos{\theta_{0}} |0\rangle U_{f(0)}|0\rangle + \sin{\theta_{0}} |1\rangle U_{f(1)}|0\rangle $$



In [ ]:
centers, probs = get_histogram(p, 0, 1, 2**2)

In [ ]:
centers

In [ ]:
probs

In [ ]:
plt.plot(np.linspace(0,1), p(np.linspace(0,1)), 'o')
plt.plot(centers, probs, 'o')

In [ ]:
qprog = Program()
qbits = qprog.qalloc(2)
#centers, probs = get_histogram(p, 0, 1, 2**1)
#f_0 = probs[0]/sum(probs)
f_0 = sum(probs[:2])

print('f_0: {}'.format(f_0))
theta_0 = np.arccos(np.sqrt(f_0))
print('theta_0: {}'.format(theta_0))
from qat.lang.AQASM import RY
qprog.apply(RY(2*theta_0),qbits[0])

#centers, probs = get_histogram(p, 0, 1, 2**2)
f_0 = probs[0]/sum(probs[0:2])
f_1 = probs[2]/sum(probs[2:])
print('f_0: {}'.format(f_0))
print('f_1: {}'.format(f_1))
theta_0 = np.arccos(np.sqrt(f_0))
theta_1 = np.arccos(np.sqrt(f_1))
from qat.lang.AQASM import X

qprog.apply(X,qbits[0])
qprog.apply(RY(2*theta_0).ctrl(),qbits[0], qbits[1])
qprog.apply(X,qbits[0])
qprog.apply(RY(2*theta_1).ctrl(),qbits[0], qbits[1])

In [ ]:
nqbits = 3
nbins = 2**nqbits
centers, probs = get_histogram(p, 0, 1, nbins)
qprog = Program()
qbits = qprog.qalloc(nqbits)
f_0 = sum(probs[:nbins//2])
print('f_0: {}'.format(f_0))
theta_0 = np.arccos(np.sqrt(f_0))
print('theta_0: {}'.format(theta_0))
from qat.lang.AQASM import RY
qprog.apply(RY(2*theta_0),qbits[0])

#iteramos sobre los demás q-bits


In [ ]:
nqbits = 5
nbins = 2**nqbits
centers, probs = get_histogram(p, 0, 1, nbins)
print('len(probs): {}'.format(len(probs)))

In [ ]:
for i in range(1, nqbits):
    #Division of the Domain for the i-th bin
    BinDivision = 2**(i+1)
    print('BinDivision: {}'.format(BinDivision))

    

In [ ]:
    step = nbins//(2**i)
    print(i,step)
    TotalBinProbability = [sum(probs[j:j+step]) for j in range(0, nbins, step)]
    LeftBinProbability = [sum(probs[j:j+2**i]) for j in range(0, nbins, step)]
    print(TotalBinProbability)
    print(LeftBinProbability)


TotalBinProbability

In [ ]:
LeftBinProbability

In [ ]:
probs

In [ ]:
#centers, probs = get_histogram(p, 0, 1, 2**1)
#f_0 = probs[0]/sum(probs)
f_0 = sum(probs[:2])

print('f_0: {}'.format(f_0))
theta_0 = np.arccos(np.sqrt(f_0))
print('theta_0: {}'.format(theta_0))
from qat.lang.AQASM import RY
qprog.apply(RY(2*theta_0),qbits[0])

In [ ]:
centers, probs = get_histogram(p, 0, 1, 2**1)

In [ ]:
centers

In [ ]:
probs

In [ ]:
centers, probs = get_histogram(p, 0, 1, 2**2)

In [ ]:
centers

In [ ]:
probs

In [ ]:
probs[0]/sum(probs[2:])

In [ ]:
sum(probs[2:])

In [ ]:
#Create the circuit from the program
circuit = qprog.to_circ()

#Display the circuit
%qatdisplay circuit

In [ ]:
#Create a Job from the circuit
job = circuit.to_job()

#Import and create the linear algebra simulator
from qat.qpus import LinAlg
linalgqpu = LinAlg()

#Submit the job to the simulator LinAlg and get the results
result = linalgqpu.submit(job)

#Print the results
for sample in result:
    print("State %s probability %s" % (sample.state, sample.probability))

In [ ]:
probs

In [ ]:
0.01+0.09

In [ ]:
probs[1]/sum(probs[:2])

In [ ]:
sum(probs[:2])*probs[0]

In [ ]:
sum(probs[2:])*probs[2]

In [ ]:
sum(probs[2:])*probs[3]

In [ ]:
probs[3]/sum(probs[2:])

In [ ]:
probs

 ## Juan

In [ ]:
nqbits = 2
nbins = 2**nqbits
a = 0
b = 1

In [ ]:
centers, probs = get_histogram(p, a, b, nbins)

In [ ]:
centers

In [ ]:
probs

In [ ]:
centers, probs = get_histogram(p, a, b, nbins)

for m in range(nqbits):
    n_parts = 2**(m+1) #Compute the number of subzones which the current state is codifying
    edges = np.array([a+(b-a)*(i)/n_parts for i in range(n_parts+1)]) #Compute the edges of that subzones
    print(edges)
    p_zones = np.array([np.sum(probs[np.logical_and(centers>edges[i],centers<edges[i+1])]) for i in range(n_parts)])
    print(p_zones)
    p_left = p_zones[[2*j for j in range(n_parts//2)]]
    print(p_left)
    p_tot = p_left + p_zones[[2*j+1 for j in range(n_parts//2)]]
    print(p_tot)
    print('** : {}'.format(p_left/p_tot))

In [ ]:
probs[0]/sum(probs)

In [ ]:
			# Compute the probabilities of each subzone by suming the probabilities of the original histogram.
			# There is no need to compute integrals since the limiting accuracy is given by the original discretization.
			# Moreover, this approach allows to handle non analytical probability distributions, measured directly from experiments
			p_zones = np.array([np.sum(probs[np.logical_and(centers>edges[i],centers<edges[i+1])]) for i in range(n_parts)])
			# Compute the probability of standing on the left part of each zone 
			p_left = p_zones[[2*j for j in range(n_parts//2)]]
			# Compute the probability of standing on each zone (left zone + right zone)
			p_tot = p_left + p_zones[[2*j+1 for j in range(n_parts//2)]]
			
			# Compute the rotation angles
			thetas = np.arccos(np.sqrt(p_left/p_tot))


In [ ]:
qprog = Program()
qbits = qprog.qalloc(5)

In [ ]:
qprog = Program()
qbits = qprog.qalloc(1)

In [ ]:
qprog = Program()
qbits = qprog.qalloc(2)
i=1
centers, probs = get_histogram(p, 0, 1, 2**i)
f_0 = probs[0]/sum(probs)
theta_0 = np.arccos(np.sqrt(f_0))
from qat.lang.AQASM import RY
qprog.apply(RY(2*theta_0),qbits[0])

i=2

centers, probs = get_histogram(p, 0, 1, 2**i)
#Calcula todos los theta
fs = []
thetas = []
for j in range(0,2**i, 2):
    #print(probs[j])
    #print(probs[j:j+2])
    f = probs[j]/sum(probs[j:j+2])
    theta = np.arccos(np.sqrt(f))
    fs.append(f)
    thetas.append(theta)


qaux = qprog.qalloc(1)
qprog.apply(X,qaux[0])
qprog.apply(RY(2*thetas[0]).ctrl(),qaux[0], qbits[1])
qprog.apply(X,qaux[0])
qprog.apply(RY(2*thetas[1]).ctrl(),qaux[0], qbits[1])



In [ ]:
from qat.lang.AQASM import QRoutine, X, RY, H

In [ ]:
i=2
routine = QRoutine()
input_wires = routine.new_wires(1) # These are our 2 input qubits
temp_wire = routine.new_wires(1) # This is our temporary qubit
routine.set_ancillae(temp_wire)
centers, probs = get_histogram(p, 0, 1, 2**i)
print(probs)
#Calcula todos los theta
fs = []
thetas = []
for j in range(0,2**i, 2):
    #print(probs[j])
    #print(probs[j:j+2])
    f = probs[j]/sum(probs[j:j+2])
    theta = np.arccos(np.sqrt(f))
    fs.append(f)
    thetas.append(theta)

routine.apply(X, temp_wire[0])
#routine.apply(H, temp_wire[1])
routine.apply(RY(2*thetas[0]).ctrl(), temp_wire, input_wires[0])
routine.apply(X, temp_wire[0])
routine.apply(RY(2*thetas[1]).ctrl(), temp_wire, input_wires[0])

In [ ]:
qprog = Program()
qbits = qprog.qalloc(2)
i=1
centers, probs = get_histogram(p, 0, 1, 2**i)
f_0 = probs[0]/sum(probs)
theta_0 = np.arccos(np.sqrt(f_0))
from qat.lang.AQASM import RY
qprog.apply(RY(2*theta_0),qbits[0])
qprog.apply(routine, qbits[1])


In [ ]:
#Create the circuit from the program
circuit = qprog.to_circ()

#Display the circuit
%qatdisplay circuit

In [ ]:
#Create a Job from the circuitç
job = circuit.to_job()

#Import and create the linear algebra simulator
from qat.qpus import LinAlg
linalgqpu = LinAlg()

#Submit the job to the simulator LinAlg and get the results
result = linalgqpu.submit(job)

#Print the results
for sample in result:
    print("State %s probability %s" % (sample.state, sample.probability))

In [ ]:
probs